In [1]:
import requests
from IPython.display import Markdown

import numpy as np
import pandas as pd
import io

**Warning!** This is a soultion. If you are looking to do these 
           [Agile Geosciences](https://agilescientific.com/blog/2020/4/16/geoscientist-challenge-thyself) 
           challenges on your own then please visit this
           [Jupyter Notebook](https://colab.research.google.com/drive/1eP68NTV-GA3R-BYUh-CUxcgYDQ5IuetS)
           to get started.

In [2]:
def get_data(url, key):
    params = {'key':my_key}
    r = requests.get(url, params)
    return r.text

def get_question(url):
    r = requests.get(url)
    return r.text

def check_answer(questionNum,answer):
    params = {'key':my_key,
              'question':questionNum,
              'answer':answer
             }
    result = requests.get(url, params)
    return result.text

## Request Challenge Description

In [3]:
url = 'https://kata.geosci.ai/challenge/true-vertical-depth' 
r = get_question(url)

Markdown(r)

# True-vertical-depth

We have some well data. Some of the wells have deviated (non-vertical) sections. All well sections are straight; the corners are not rounded. Each well record is represented by 7 numbers:

     x   y   d  k   i  a    t
    123,654,24,450,12,254,1200
    
These numbers are (_x_, _y_) of the surface location, the elevation of the datum _d_, the kick-off point _k_ (from the datum), the inclination _i_ and azimuth _a_ of the deviated section, and the total depth _t_ (measured along the hole from the datum).

Some wells have an unknown datum, denoted by 'UNK'; you should ignore them.

We wish to know:

1. What is the highest datum?
2. What is the greatest offset distance reached, **to the nearest metre**? 
3. What is the greatest true vertical depth reached, **to the nearest metre**?
4. At a TVDSS of -2000 m, how many wells have **an absolute _y_-offset** of more than 100 m?

Note the following assumptions and definitions:

- *Measured depth* is measured along the well path, starting at the top, and is always positive.
- Elevations are positive above sea-level, and negative below sea-level.
- *True vertical depth sub-sea*, or TVDSS, is the elevation along the well path.
- The *offset distance* is the horizontal distance from the surface location to the bottomhole location.
- The section of the well above the *kick-off point* is vertical.
- A kick-off point of -999.25 indicates that there is no deviated section.
- The 'deviated' section of the well below the kick-off point is straight but inclined at _i_ degrees off the vertical.
- On a map, the deviated section of the well is oriented towards the direction _a_ degrees clockwise from North.

This diagram might help. Then again, it might not.

<img src="https://kata.geosci.ai/static/tvd.svg">


## Example wells

Your questions relate to the collection of wells. In this example, we'll look at just one well:

    212,32,68,322,8,295,2630

- The datum of this well is **68** m above sea-level.
- The offset distance at the bottom of this well is about 321.21 m, or **321** to the nearest m.
- The TVDSS at the bottom of this well is -2539.54 m, or **-2540** to the nearest m.
- At a TVDSS of -2000 m, this well has a y-offset of 103.7 m and therefore would be counted for question 4.

Let's look at another well.

    416,232,89,-999.25,-999.25,-999.25,1989

- The datum of this well is **89** m above sea-level.
- The offset distance at the bottom of this well is **0** m; it is not deviated.
- The TVDSS at the bottom of this well is **-1900** m.
- This well does not reach -2000 m.


## A quick reminder how this works

You can retrieve your data by choosing any Python string as a **`<KEY>`** and substituting here:

    https://kata.geosci.ai/challenge/true-vertical-depth?key=<KEY>
                                                             ^^^^^
                                                             any old string you like

To answer question 1, make a request like:

    https://kata.geosci.ai/challenge/true-vertical-depth?key=<KEY>&question=1&answer=123
                                                             ^^^^^          ^        ^^^
                                                             your key       Q        your answer

[Complete instructions at kata.geosci.ai](https://kata.geosci.ai/challenge)

----

© 2020 Agile Scientific, licensed CC-BY


## My solution

Let's enter a seed phrase and get the data.

In [4]:
my_key = 'armstrys'

## Input
r = get_data(url, my_key)

r[:400]

'21053,35243,256,418,26,126,898\n19970,10094,191,419,3,151,1550\n24111,5960,256,442,20,113,2275\n15233,35942,145,624,29,201,1178\n15952,17874,100,444,14,173,1662\n7064,10047,199,354,1,117,2261\n21782,6773,189,541,8,266,2090\n30249,10944,274,484,2,132,1378\n26042,28846,212,540,2,19,1970\n9065,12939,142,671,17,288,3195\n17170,29056,148,522,6,56,1838\n7887,10988,171,608,5,270,1399\n21206,32834,246,581,26,166,1223'

## Making the text meaningful
The input text is relatively tame in this case. We can just pass the text through `io.StringIO` and into `pandas.read_csv`. We'll then eliminate some unknown and `NaN` values.

In [5]:
cols = [
        'X',
        'Y',
        'Datum',
        'Kickoff',
        'Inclination',
        'Azimuth',
        'MD'
        ]

data = pd.read_csv(io.StringIO(r),sep=',',names=cols, na_values=-999.25) #load text to pandas
data = data[~(data['Datum'] == 'UNK')]
data['Datum'] = data['Datum'].astype(int)
data['Inclination'].fillna(0,inplace=True)
data['Kickoff'].fillna(0,inplace=True)
    
data.head()

,X,Y,Datum,Kickoff,Inclination,Azimuth,MD
0,21053,35243,256,418.0,26.0,126.0,898
1,19970,10094,191,419.0,3.0,151.0,1550
2,24111,5960,256,442.0,20.0,113.0,2275
3,15233,35942,145,624.0,29.0,201.0,1178
4,15952,17874,100,444.0,14.0,173.0,1662


## Question 1
For question 1, all we need to do is find the max of our `Datum` column.'

In [7]:
answer1 = data['Datum'].max()

print(f'The highest elevation well is at {answer1} m elevation.\n')

## Check
questionNum = 1
result = check_answer(questionNum,answer1)
print(f'Your answer is {result.lower()}!')

The highest elevation well is at 333 m elevation.

Your answer is correct!


## Question 2
The horizontal offset of each well can be calculated using simple trigonometry from the length of the deviated section of the well and the deviation inclination. Azimuth is irrelevant here since we are calculating radial offset.

In [8]:
# length of lower section - possibly deviated
lowerLength = data['MD'] - data['Kickoff']

# calculating horizontal offset down to TD
data['TD_Offset'] = np.sin(np.radians(data['Inclination'])) * lowerLength

answer2 = round(data['TD_Offset'].max())

data[['Kickoff','MD','Inclination','TD_Offset']].sort_values('TD_Offset', ascending=False).head()


,Kickoff,MD,Inclination,TD_Offset
516,519.0,3523,26.0,1316.866925
639,427.0,3200,28.0,1301.844644
687,508.0,3147,29.0,1279.412588
658,401.0,3192,27.0,1267.087485
213,582.0,3268,28.0,1261.000618


In [9]:
print(f'The maximum offset is {answer2} m.\n')

## Check
questionNum = 2
result = check_answer(questionNum,answer2)
print(f'Your answer is {result.lower()}!')

The maximum offset is 1317 m.

Your answer is correct!


## Question 3
For question 3, we'll be calculating the total vertical depth (subsea). This calculation will be similar to the calcation for question 2, but will also need to take into account the datum of the well. The question asks for the offset at -2000 m subsea, but you can adjust the slider to see how things change.

In [10]:

# length of lower section - possibly deviated
lowerLength = data['MD'] - data['Kickoff']

# use trig to calculate vertical component of deviation
lowerVertical = np.cos(np.radians(data['Inclination'])) * lowerLength

# add upper and lower sections of well and adjust for datum
data['TVDSS'] = data['Datum'] - data['Kickoff'] - lowerVertical 

answer3 = round(data['TVDSS'].min())

data[['Datum','Kickoff','MD','Inclination','TVDSS']].sort_values('TVDSS', ascending=True).head()


,Datum,Kickoff,MD,Inclination,TVDSS
100,227,608.0,3538,4.0,-3303.862667
925,153,599.0,3637,20.0,-3300.786182
452,116,391.0,3444,8.0,-3298.288414
579,187,408.0,3459,7.0,-3249.258309
720,153,653.0,3498,17.0,-3220.687031


In [11]:
print(f'The largest total vertical depth is {answer3} m.\n')

## Check
questionNum = 3
result = check_answer(questionNum,answer3)
print(f'Your answer is {result.lower()}!')

The largest total vertical depth is -3304 m.

Your answer is correct!


## Question 4
We need to drop some wells. Wells that don't reach 2000 m tvdss won't be counted, and neither will wells that have no deviation. Null azimuths will break the function and we know those wells won't have a y-offset > 100 m anyway.

In [12]:
sliceSS = -2000

# Remove wells that don't hit the depth slice or don't kave a kickoff point above
data_subset = data[data['TVDSS'] < sliceSS].dropna()
mask_devAbove2000 = ~((data_subset['Datum'] - data_subset['Kickoff']) <= sliceSS)
data_subset = (data_subset[mask_devAbove2000].dropna())

# Calculate the Y offset between kickoff and depth slice!
upperSectionSS = data_subset['Datum'] - data_subset['Kickoff']
lowerVertical = upperSectionSS - sliceSS
offset2000 = np.tan(np.radians(data_subset['Inclination'])) * lowerVertical
data_subset['YOffset2000'] = np.abs(np.cos(np.radians(data_subset['Azimuth'])) * offset2000)

answer4 = len(data_subset.loc[data_subset['YOffset2000']>100,'YOffset2000'])

data_subset[['Datum','Kickoff','MD','Inclination','TVDSS','YOffset2000']].sort_values('YOffset2000', ascending=False).head()


,Datum,Kickoff,MD,Inclination,TVDSS,YOffset2000
410,227,345.0,2720,29.0,-2195.221804,1024.042936
824,195,401.0,2607,29.0,-2135.411074,928.380793
639,184,427.0,3200,28.0,-2691.413675,927.249986
673,243,430.0,2841,27.0,-2335.216730,923.206904
835,177,462.0,2654,29.0,-2202.166398,922.401952


In [13]:
print(f'There are {answer4} wells with y-offset greater than 100m at {sliceSS} m depth.\n')

## Check
questionNum = 4
result = check_answer(questionNum,answer4)
print(f'Your answer is {result.lower()}')

There are 283 wells with y-offset greater than 100m at -2000 m depth.

Your answer is correct! the next challenge is not ready yet. check this url again later.
